## Import library

In [1]:
import datetime 
from datetime import timedelta 
import os
import glob
import numpy as np
import pandas as pd
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import roc_curve
from torch import nn, optim

import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from itertools import cycle
from tqdm.notebook import tqdm
import re
from sklearn.metrics import roc_auc_score, plot_roc_curve

import matplotlib.pyplot as plt
import seaborn
from collections import defaultdict

from sklearn import svm

## Data load

In [2]:
# df_israel_covid 이스라엘 데이터셋
# corona_isolation_per_day_ver_0059 일일 코로나 격리자 수 
# time_series_covid19_confirmed_global 누적 확진자 수 

base_path = './opendataset'
df_israel_covid = pd.read_csv(f'{base_path}/israel_covid_large_2.csv')
df_israel_isolation_per_day = pd.read_csv(f'{base_path}/corona_isolation_per_day_ver_0069.csv')
df_timeseries_confirm = pd.read_csv(f'{base_path}/time_series_covid19_confirmed_global.csv')

/home/md/anaconda3/envs/flu_screening/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_timeseries_confirm.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/28/21,8/29/21,8/30/21,8/31/21,9/1/21,9/2/21,9/3/21,9/4/21,9/5/21,9/6/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,152960,152960,153148,153220,153260,153306,153375,153375,153375,153534
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,144079,144847,145333,146387,147369,148222,149117,150101,150997,151499
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,194671,195162,195574,196080,196527,196915,197308,197659,198004,198313
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,15025,15025,15032,15033,15046,15052,15055,15055,15055,15069
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,47079,47168,47331,47544,47781,48004,48261,48475,48656,48790


## preprocessing

In [4]:
# 누적 확진자수를 2021-08-17 와 같은 포멧으로 전처리해주는 과정

rows = []
for idx, values in enumerate(df_timeseries_confirm[df_timeseries_confirm['Country/Region'] == 'Israel'].iteritems()):
    if idx < 4: continue
    year = f"20{values[0][-2:]}"
    month = values[0][:values[0].index('/')]
    month = f'0{month}' if len(month) == 1 else month
    day = values[0][values[0].index('/') + 1 : values[0].rindex('/')]
    day = f'0{day}' if len(day) == 1 else day
    date = f'{year}-{month}-{day}'
    confirmed_num = values[1].item()
    rows.append([date, confirmed_num])
df_numsum_confirmed = pd.DataFrame(rows, columns=['confirmed_date', 'confirmed_numsum'])

In [5]:
# 누적 확진자를 통해 일일 확진자 구하기

df_numsum_confirmed['confirmed_num'] = df_numsum_confirmed['confirmed_numsum'] - df_numsum_confirmed.shift(1)['confirmed_numsum']
df_numsum_confirmed = df_numsum_confirmed.dropna()
df_numsum_confirmed['confirmed_num'] = df_numsum_confirmed['confirmed_num'].astype(int)

In [6]:
# 일일 확진자수가 음수인 경우 그 전날 확진자를 가져다 쓰는 과정

for idx, row in df_numsum_confirmed[df_numsum_confirmed['confirmed_num'] < 0].iterrows():
    print("before", df_numsum_confirmed.loc[idx,'confirmed_num'])
    df_numsum_confirmed.loc[idx,'confirmed_num'] = df_numsum_confirmed.loc[idx-1,'confirmed_num']
    print("after", df_numsum_confirmed.loc[idx,'confirmed_num'])
    print()

before -28
after 28

before -53
after 34



In [7]:
df_numsum_confirmed.tail(5)

,confirmed_date,confirmed_numsum,confirmed_num
589,2021-09-02,1096881,13900
590,2021-09-03,1104971,8090
591,2021-09-04,1112964,7993
592,2021-09-05,1117596,4632
593,2021-09-06,1117596,0


In [8]:
df_israel_covid = df_israel_covid[df_israel_covid['test_date'] < '2021-09-01']
df_israel_covid = df_israel_covid[df_israel_covid['test_date'] >= '2021-06-01']
df_israel_isolation_per_day = df_israel_isolation_per_day[df_israel_isolation_per_day['date'] < '2021-09-01']
df_israel_isolation_per_day = df_israel_isolation_per_day[df_israel_isolation_per_day['date'] >= '2021-06-01']
df_numsum_confirmed = df_numsum_confirmed[df_numsum_confirmed['confirmed_date'] < '2021-09-01']
df_numsum_confirmed = df_numsum_confirmed[df_numsum_confirmed['confirmed_date'] >= '2021-06-01']

In [9]:
df_numsum_confirmed.tail(5)

,confirmed_date,confirmed_numsum,confirmed_num
583,2021-08-27,1034623,6352
584,2021-08-28,1045800,11177
585,2021-08-29,1051609,5809
586,2021-08-30,1061488,9879
587,2021-08-31,1066352,4864


In [10]:
df_israel_isolation_per_day['join_date'] = (pd.to_datetime(df_israel_isolation_per_day['date']) + timedelta(days=1)).dt.strftime('%Y-%m-%d')

# 요일을 숫자로 기록한 컬럼 하나 만듬
df_israel_covid['weekday'] = pd.to_datetime(df_israel_covid['test_date']).dt.dayofweek

In [11]:
df_israel_covid['test_date'].describe()

count        1143373
unique            92
top       2021-08-23
freq           22536
Name: test_date, dtype: object

In [12]:
df_israel_isolation_per_day['join_date'].describe()

count             92
unique            92
top       2021-07-12
freq               1
Name: join_date, dtype: object

In [13]:
df_numsum_confirmed.describe()

,confirmed_numsum,confirmed_num
count,9.200000e+01,92.000000
mean,8.838310e+05,2466.054348
std,6.347984e+04,3121.443944
min,8.395110e+05,0.000000
25%,8.403892e+05,140.250000
50%,8.505360e+05,908.000000
75%,9.019628e+05,4274.500000
max,1.066352e+06,12113.000000


In [14]:
# 기존 이스라엘 데이터에 격리자 컬럼을 넣어주는 과정

df_israel_covid = pd.merge(df_israel_covid, df_israel_isolation_per_day, how='left', left_on='test_date', right_on='join_date')
print("After merge:", df_israel_covid.shape)
df_israel_covid = df_israel_covid.sort_values('test_date')
df_israel_covid.head(5)

After merge: (1143373, 17)


,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication,weekday,date,isolated_today_contact_with_confirmed,isolated_today_abroad,new_contact_with_confirmed,new_from_abroad,join_date
1143372,2021-06-01,0,0,0,0,0,שלילי,NaN,NaN,Other,1,NaN,NaN,NaN,NaN,NaN,NaN
1140387,2021-06-01,0,0,0,0,0,שלילי,NaN,NaN,Other,1,NaN,NaN,NaN,NaN,NaN,NaN
1140386,2021-06-01,0,0,0,0,0,שלילי,NaN,NaN,Other,1,NaN,NaN,NaN,NaN,NaN,NaN
1140385,2021-06-01,0,0,0,0,0,שלילי,NaN,NaN,Other,1,NaN,NaN,NaN,NaN,NaN,NaN
1140384,2021-06-01,0,0,0,0,0,שלילי,NaN,NaN,Other,1,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# 기존 이스라엘 데이터에 일일 확진자랑 일일 누적 확진자 컬럼을 넣어주는 과정

df_israel_covid = pd.merge(df_israel_covid, df_numsum_confirmed, how='left', left_on='test_date', right_on='confirmed_date')
print("After merge accumulated: ", df_israel_covid.shape[0])

After merge accumulated:  1143373


In [16]:
head_ne = df_israel_covid[df_israel_covid['head_ache'] == 0]
head_po = df_israel_covid[df_israel_covid['head_ache'] == 1]
print(head_ne.shape[0]+head_po.shape[0])

1143373


null 값이 있는지 확인

In [17]:
# 없어야함.
df_israel_covid[df_israel_covid['confirmed_num'].isnull()].shape

(0, 20)

## 번역하기

In [18]:
def gender_map(gender):
    if gender == 'נקבה': return 1 #'Female'
    elif gender == 'זכר': return 0 #'Male'
    return -1

In [19]:
def corona_result_map(corona_result):
    if corona_result == 'אחר': return -1 # 'Other'
    elif corona_result == 'שלילי': return 0 # 'Negative'
    elif corona_result == 'חיובי': return 1 # 'Positive'
    return ''

In [20]:
def age_60_and_above_map(age_60_and_above):
    if age_60_and_above == 'Yes': return 1
    elif age_60_and_above == 'No': return 0
    return -1

In [21]:
def test_indication_map(test_indication):
    if test_indication == 'Other': return 0
    elif test_indication == 'Abroad': return 1
    elif  test_indication == 'Contact with confirmed': return 2
    else: return -1

In [22]:
df_israel_covid['gender'] = df_israel_covid['gender'].apply(gender_map)
df_israel_covid['corona_result'] = df_israel_covid['corona_result'].apply(corona_result_map)
df_israel_covid['age_60_and_above'] = df_israel_covid['age_60_and_above'].apply(age_60_and_above_map)
df_israel_covid['test_indication'] = df_israel_covid['test_indication'].apply(test_indication_map)

In [23]:
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data)).astype(int)
    test_set_size = int(len(data) * test_ratio)
    
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    
    print(len(data))
    print(test_indices.dtype)
    print(train_indices.dtype)
    
    test_data = data[:test_set_size]
    train_data = data[test_set_size:]
    test_data = data[test_indices]
    train_data = data[train_indices]
    
    return train_data, test_data

In [24]:
def split_input_gt(non_split_data):
    splited_input = non_split_data[:, :-1]
    splited_gt = non_split_data[:, -1]
    
    return splited_input, splited_gt

## model input preprocessing

In [25]:
# other 빼버림
df_israel_covid_without_o = df_israel_covid[df_israel_covid['corona_result'] != -1]

In [26]:
df_israel_covid_without_o.shape[0]

1091291

In [27]:
# 성별 없는 사람들 빼버림
df_israel_covid_without_o = df_israel_covid_without_o[df_israel_covid_without_o['gender'] != -1]

In [28]:
df_israel_covid_without_o.shape[0]

881077

In [29]:
df_israel_covid_without_o = df_israel_covid_without_o.sort_values(['test_date']).reset_index(drop=True)

In [30]:
df_israel_covid_without_o['test_date'].min()

'2021-06-05'

In [31]:
# # week_num 이라는 칼럼을 새로 만들었다. 1년에 52주라 
# def date_to_weeknum_map(test_date):
#     week_num_st_idx = 52
#     year_last_num = int(test_date[2:4])
#     week_num = datetime.date(int(test_date[:4]), int(test_date[5:7]), int(test_date[8:])).isocalendar()[1]
#     return week_num_st_idx * year_last_num + week_num

In [32]:
# df_israel_covid_without_o['week_num'] = df_israel_covid_without_o['test_date'].apply(date_to_weeknum_map)

# Add 2nd degree data
추가 feature
- new_contact_with_confirmed
- positive_ratio
- weekday_ratio
- confirmed_ratio

In [33]:
df_israel_covid_without_o['new_contact_with_confirmed'] = df_israel_covid_without_o['new_contact_with_confirmed'].astype(int)

In [34]:
# 36903이 뭔지?
df_israel_covid_without_o['contact'] = df_israel_covid_without_o['new_contact_with_confirmed'] / df_israel_covid_without_o['new_contact_with_confirmed'].max()

In [35]:
df_confirm_by_sampling = df_israel_covid_without_o[df_israel_covid_without_o['corona_result'] == 1].groupby('test_date').count().reset_index()[['test_date', 'date']]
df_confirm_by_sampling = df_confirm_by_sampling.rename(columns={'test_date': 'yesterday', 'date': 'positive_cnt'})

In [36]:
df_israel_covid_without_o = pd.merge(df_israel_covid_without_o, df_confirm_by_sampling, how='left', left_on='date', right_on='yesterday')

In [37]:
df_israel_covid_without_o.shape[0]

881077

In [38]:
df_israel_covid_without_o = df_israel_covid_without_o[~df_israel_covid_without_o['positive_cnt'].isnull()]

In [39]:
df_israel_covid_without_o['positive_cnt'] = df_israel_covid_without_o['positive_cnt'].astype(int)

In [40]:
df_israel_covid_without_o['positive_ratio'] = df_israel_covid_without_o['positive_cnt'] / df_israel_covid_without_o['positive_cnt'].max()

In [41]:
df_israel_covid_without_o['weekday_ratio'] = df_israel_covid_without_o['weekday'] / 6

In [42]:
df_israel_covid_without_o['confirmed_ratio'] = df_israel_covid_without_o['confirmed_num'] / df_israel_covid_without_o['confirmed_num'].max()

In [43]:
df_israel_covid_without_o.columns

Index(['test_date', 'cough', 'fever', 'sore_throat', 'shortness_of_breath',
       'head_ache', 'corona_result', 'age_60_and_above', 'gender',
       'test_indication', 'weekday', 'date',
       'isolated_today_contact_with_confirmed', 'isolated_today_abroad',
       'new_contact_with_confirmed', 'new_from_abroad', 'join_date',
       'confirmed_date', 'confirmed_numsum', 'confirmed_num', 'contact',
       'yesterday', 'positive_cnt', 'positive_ratio', 'weekday_ratio',
       'confirmed_ratio'],
      dtype='object')

In [44]:
# 여기서 내가 사용할 feature 고르기
# df_input = = df_israel_covid_without_o[['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache', 'corona_result']]
df_input = df_israel_covid_without_o[['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache', 'age_60_and_above', 'gender', 'confirmed_ratio', 'weekday_ratio', 'corona_result']]
raw_datas = df_input.values

In [45]:
df_input.describe()

,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender,confirmed_ratio,weekday_ratio,corona_result
count,862295.00000,862295.000000,862295.000000,862295.000000,862295.000000,862295.000000,862295.000000,862295.000000,862295.000000,862295.000000
mean,0.01470,0.014740,0.006190,0.001066,0.011723,-0.038449,0.541671,0.336005,0.440874,0.057483
std,0.12035,0.120509,0.078436,0.032629,0.107638,0.548196,0.498261,0.264007,0.334477,0.232763
min,0.00000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.015933,0.000000,0.000000
25%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103525,0.166667,0.000000
50%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.260547,0.333333,0.000000
75%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.524395,0.666667,0.000000
max,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [46]:
not_test_datas, test_datas = split_train_test(raw_datas, test_ratio=0.2)
train_datas, valid_datas = split_train_test(not_test_datas, test_ratio=0.2)

train_input, train_gt = split_input_gt(train_datas)
valid_input, valid_gt = split_input_gt(valid_datas)
test_input, test_gt = split_input_gt(test_datas)

862295
int64
int64
689836
int64
int64


In [47]:
print(train_input.shape, train_gt.shape, valid_input.shape, valid_gt.shape, test_input.shape, test_gt.shape)

(551869, 9) (551869,) (137967, 9) (137967,) (172459, 9) (172459,)


In [48]:
print("# of train data positive: ", train_gt.sum())
print("# of train data negative: ", train_gt.shape[0] - train_gt.sum())
print("ratio of P/N: ", train_gt.sum()/(train_gt.shape[0] - train_gt.sum()))
print("")

print("# of valid data positive: ", valid_gt.sum())
print("# of valid data negative: ", valid_gt.shape[0] - valid_gt.sum())
print("ratio of P/N: ", valid_gt.sum()/(valid_gt.shape[0] - valid_gt.sum()))
print("")

print("# of test data positive: ", test_gt.sum())
print("# of test data negative: ", test_gt.shape[0] - test_gt.sum())
print("ratio of P/N: ", test_gt.sum()/(test_gt.shape[0] - test_gt.sum()))

# of train data positive:  31754.0
# of train data negative:  520115.0
ratio of P/N:  0.06105188275669804

# of valid data positive:  7883.0
# of valid data negative:  130084.0
ratio of P/N:  0.06059930506441991

# of test data positive:  9930.0
# of test data negative:  162529.0
ratio of P/N:  0.06109678888075359


In [49]:
class IsraelDataset(Dataset):
    def __init__(self, input_data, gt, transform=None, target_transform=None):
        self.input_data = input_data
        self.gt = gt
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.input_data)
    
    def __getitem__(self, idx):
        return self.input_data[idx], self.gt[idx]

In [50]:
# def plot_roc_curve(fper, tper):
#     plt.plot(fper, tper, color='red', label='ROC')
#     plt.plot([0, 1], [0, 1], color='green', linestyle='--')
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Receiver Operating Characteristic Curve')
#     plt.legend()
#     plt.show()

In [51]:
def display_performance(clf, pred, valid_input, valid_gt):
    pred = np.array(pred)

    print('accuracy %.4f' % (accuracy_score(pred, valid_gt)))
    print('precision %.4f' % (precision_score(pred, valid_gt)))
    print('recall %.4f' % (recall_score(pred, valid_gt)))
    print('f1-score %.4f' % (f1_score(pred, valid_gt)))
    print(confusion_matrix(pred, valid_gt))
    plot_roc_curve(clf, valid_input, valid_gt)

In [52]:
training_data = IsraelDataset(torch.Tensor(train_input), torch.Tensor(train_gt.reshape(-1, 1)))
valid_data = IsraelDataset(torch.Tensor(valid_input), torch.Tensor(valid_gt.reshape(-1, 1)))

train_dataloader = DataLoader(training_data, batch_size = 2, shuffle=False)
valid_dataloader = DataLoader(valid_data, batch_size = 2, shuffle=False)

valid_input.shape

(137967, 9)

In [53]:
for val_inputs, val_labels in valid_dataloader:
    break
print(val_inputs.shape)

print(val_inputs.shape)

torch.Size([2, 9])
torch.Size([2, 9])


In [54]:
import torch
from torch import Tensor
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Optional


def conv3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1) -> nn.Conv1d:
    return nn.Conv1d(in_planes, out_planes,
                     kernel_size=3,
                     stride=stride, padding=1,
                     groups=groups, bias=False,
                     )


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(self, inplanes: int, planes: int, stride: int = 1, ) -> None:
        super(BasicBlock, self).__init__()
        norm_layer = nn.BatchNorm1d
        self.conv1 = conv3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes, n_channels=64, dropout=0.2):

        super(ResNet, self).__init__()

        self.inplanes = 1
        self.outplanes = n_channels
        self.conv1 = nn.Conv1d(self.inplanes, self.outplanes, kernel_size=7, stride=1, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(self.outplanes)
        self.relu = nn.ReLU(inplace=True)

        # Update inplanes for the next layer
        self.inplanes = self.outplanes

        # ResNet blocks
        self.layer1 = self._make_layer(block, n_channels, layers[0])
        self.layer2 = self._make_layer(block, n_channels, layers[1], stride=1,)
        self.layer3 = self._make_layer(block, n_channels, layers[2], stride=1,)
        self.layer4 = self._make_layer(block, n_channels, layers[3], stride=1,)
        self.avgpool = nn.AdaptiveAvgPool1d((1, ))
        
        self.dropout = nn.Dropout(p=dropout)
        self.fc = nn.Linear(self.inplanes * block.expansion, num_classes)

    def _make_layer(self, block, planes: int, blocks: int, stride: int = 1, dilate: bool = False):
        layers = []
        layers.append(block(self.inplanes, planes, stride))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, stride))

        return nn.Sequential(*layers)

    def forward(self, x: Tensor):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [55]:
model = ResNet(BasicBlock, [2, 2, 2, 2], 1, n_channels=64, dropout=0.9)

In [56]:
model

ResNet(
  (conv1): Conv1d(1, 64, kernel_size=(7,), stride=(1,), padding=(3,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64

In [57]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(device)

cuda


In [58]:
model = model.to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001) # lr 크기좀 늘려보기

In [59]:
def plot_roc_curve_for_dnn(fper, tper):
    plt.plot(fper, tper, color='red', label='ROC')
    plt.plot([0, 1], [0, 1], color='green', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend()
    plt.show()

In [60]:
def show_progress(train_history, val_history):
    plt.plot(train_history)
    plt.plot(val_history)
    plt.legend(['train_loss', 'val_loss'])
    plt.show()

In [ ]:
train_running_loss = 0.0
num_epochs = 30

train_history = []
val_history = []

for epoch in tqdm(range(num_epochs)):
    model.train()
    train_losses = []
    for train_inputs, train_labels in tqdm(train_dataloader):
        train_inputs = torch.unsqueeze(train_inputs, 1)
        train_inputs = train_inputs.to(device)
        train_labels = train_labels.to(device)

        optimizer.zero_grad()
        
        train_outputs = model(train_inputs)
        train_outputs = nn.functional.sigmoid(train_outputs)
        train_loss = criterion(train_outputs, train_labels)
        train_loss.backward()
        optimizer.step()
        train_losses.append(train_loss)
    
    model.eval()
    
    val_losses = []
    val_all_outputs = []
    val_all_labels = []
    for val_inputs, val_labels in valid_dataloader:
        val_inputs = torch.unsqueeze(val_inputs, 1)
        val_outputs = model(torch.Tensor(val_inputs).to(device))
        val_outputs = nn.functional.sigmoid(val_outputs)

        val_loss_on_batch = criterion(val_outputs, val_labels.to(device))
        val_losses.append(val_loss_on_batch)
        val_all_outputs.append(val_outputs.cpu().detach().numpy())
        val_all_labels.append(val_labels.cpu().detach().numpy())

    val_all_outputs = np.concatenate(val_all_outputs, axis=0)
    val_all_labels = np.concatenate(val_all_labels, axis=0)
    print(val_all_outputs[:5], val_all_labels[:5])
    auc = roc_auc_score(val_all_labels, val_all_outputs)    
    

    total_val_loss = sum(val_losses) / len(valid_dataloader)
    total_train_loss = sum(train_losses) / len(train_dataloader)
    print('Done. Train loss:{}, Validation loss: {} VAL_AUC:{}'.format(total_train_loss, total_val_loss, auc))
    
    fper, tper, thresholds = roc_curve(val_all_labels, val_all_outputs)
    plot_roc_curve_for_dnn(fper, tper)
    
    train_history.append(total_train_loss.cpu().detach().numpy())
    val_history.append(total_val_loss.cpu().detach().numpy())
    
    show_progress(train_history, val_history)
    
    

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/275935 [00:00<?, ?it/s]

/home/md/anaconda3/envs/flu_screening/lib/python3.8/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
